# Эксперимент 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from optimization import barrier_method_lasso

sns.set(style='whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
np.random.seed(2022)
A = np.random.rand(10000, 500)
b = np.random.rand(10000)
reg_coef = 1.0
x_0, u_0 = np.zeros(500), 10 * np.ones(500)

In [ ]:
def experiment_a(A, b, reg_coef, x_0, u_0, params, graph_type, figsize):
    plt.figure(figsize=figsize)
    formats = ['X:', 's-', '^--', 'D-.'] # <= 4 lines
    for param, fmt in zip(params['plot']['grid'], formats):
        kwargs = {params['fix']['name']: params['fix']['value'],
                  params['plot']['name']: param}
        _, message, history = barrier_method_lasso(A, b, reg_coef, x_0, u_0, trace=True, **kwargs)
        print(message) # assert message == 'success'
        x = np.arange(len(history['duality_gap'])) if graph_type == 'i' else np.array(history['time']) * 1000
        y = history['duality_gap']
        plt.plot(x, y, fmt, label=f"{params['plot']['name']} = {param}", linewidth=3, markersize=10)

    plt.title(f"{params['fix']['name']} = {params['fix']['value']}", fontsize=14)
    plt.xlabel('iterations' if graph_type == 'i' else 'time (ms)', fontsize=14)
    plt.ylabel('duality gap', fontsize=14)
    plt.yscale('log')
    plt.legend(fontsize=12)
    plt.savefig(f"exp_a_{params['fix']['name']}_{params['fix']['value']}_{graph_type}.pdf", bbox_inches='tight')

In [ ]:
gammas = [10, 100, 1000, 10000]
tolerances = [1e-5, 1e-8, 1e-10, 1e-12]

params = {'fix':  {'name': 'gamma', 'value': 10}, # default value
          'plot': {'name': 'tolerance_inner', 'grid': tolerances}}
experiment_a(A, b, reg_coef, x_0, u_0, params, 'i', (6, 4.5))
experiment_a(A, b, reg_coef, x_0, u_0, params, 't', (6, 4.5))

params = {'fix':  {'name': 'tolerance_inner', 'value': 1e-8}, # default value
          'plot': {'name': 'gamma', 'grid': gammas}}
experiment_a(A, b, reg_coef, x_0, u_0, params, 'i', (6, 4.5))
experiment_a(A, b, reg_coef, x_0, u_0, params, 't', (6, 4.5))

# Эксперимент 2

In [ ]:
def configure_plot(m, n, xlabel):
    plt.title(f"m = {m}, n = {n}", fontsize=14)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel('duality gap', fontsize=14)
    plt.yscale('log')
    plt.legend(fontsize=12)
    plt.savefig(f"exp_b_{m}_{n}_{xlabel[0]}.pdf", bbox_inches='tight')
    plt.clf()


def experiment_b(datasets, regcoefs, figsize):
    plt.figure(figsize=figsize)
    formats = ['X:', 's-', '^--', 'D-.'] # 4 lines
    for A, b, x_0, u_0 in datasets:
        data_x, data_y = [], []
        for reg_coef in regcoefs:
            _, message, history = barrier_method_lasso(A, b, reg_coef, x_0, u_0, trace=True)
            print(message) # assert message == 'success'
            data_x.append(np.array(history['time']) * 1000)
            data_y.append(history['duality_gap'])
        for y, reg_coef, fmt in zip(data_y, regcoefs, formats):
            plt.plot(np.arange(len(y)), y, fmt, label=f"reg_coef = {reg_coef}", linewidth=3, markersize=10)
        configure_plot(b.size, x_0.size, 'iterations')
        for x, y, reg_coef, fmt in zip(data_x, data_y, regcoefs, formats):
            plt.plot(x, y, fmt, label=f"reg_coef = {reg_coef}", linewidth=3, markersize=10)

In [ ]:
m_dims = [1000, 5000, 10000]
n_dims = [50, 100, 1000]
regcoefs = [0.1, 1, 10, 200]
datasets = []

np.random.seed(2022)
for m in m_dims:
    for n in n_dims:
        A = np.random.rand(m, n)
        b = np.random.rand(m)
        x_0, u_0 = np.zeros(n), 10 * np.ones(n)
        datasets.append((A, b, x_0, u_0))

experiment_b(datasets, regcoefs, (6, 4.5))